# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
cities=pd.read_csv("../WeatherPY/weather_output.csv")
cities.head()

,Unnamed: 0,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Long,Temp,Wind Speed,Max Temp
0,0,804,hambantota,93,LK,1611965531,82,6.1241,81.1185,24.15,3.85,24.15
1,1,600,nokia,90,FI,1611965531,93,61.4667,23.5000,-7.66,4.12,-7.22
2,2,800,busselton,0,AU,1611965532,69,-33.6500,115.3333,19.94,2.24,20.56
3,3,802,albany,40,US,1611965532,69,42.6001,-73.9662,-11.78,0.45,-10.00
4,4,804,punta arenas,90,CL,1611965532,66,-53.1500,-70.9167,8.94,7.20,9.00


In [51]:
cities["Max Temp"].max()

33.0

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)

In [16]:
locations= cities[["Lat", "Long"]]


fig =gmaps.figure()

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
# this works getting location markers to show
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
 #adding the heat map
locations= cities[["Lat", "Long"]]

rating=cities['Humidity']

# plot heatmap
fig = gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations,weights=rating,dissipating=False,max_intensity=300,
                                point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
cities

,Unnamed: 0,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Long,Temp,Wind Speed,Max Temp
0,0,804,hambantota,93,LK,1611965531,82,6.1241,81.1185,24.15,3.85,24.15
1,1,600,nokia,90,FI,1611965531,93,61.4667,23.5000,-7.66,4.12,-7.22
2,2,800,busselton,0,AU,1611965532,69,-33.6500,115.3333,19.94,2.24,20.56
3,3,802,albany,40,US,1611965532,69,42.6001,-73.9662,-11.78,0.45,-10.00
4,4,804,punta arenas,90,CL,1611965532,66,-53.1500,-70.9167,8.94,7.20,9.00
...,...,...,...,...,...,...,...,...,...,...,...,...
573,573,500,kalangala,87,UG,1611965657,81,-0.3089,32.2250,22.16,3.84,22.16
574,574,804,rantauprapat,100,ID,1611965412,93,2.1000,99.8333,22.04,0.77,22.04
575,575,804,luganville,96,VU,1611965658,69,-15.5333,167.1667,29.70,3.88,29.70
576,576,802,pitimbu,40,BR,1611965659,83,-7.4706,-34.8086,27.00,4.12,27.00


In [59]:
#A max temperature lower than 80 degrees but higher than 65.
#Wind speed less than 5 mph.
#Zero cloudiness.

ideal_temp = cities.loc[(cities["Max Temp"] <90) &(cities["Max Temp"] >25) & (cities["Wind Speed"] <10) & (cities["Cloudiness"]<6)]
ideal_temp

,Unnamed: 0,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Long,Temp,Wind Speed,Max Temp
11,11,800,carnarvon,1,AU,1611965534,33,-24.8667,113.6333,33.00,6.69,33.00
27,27,500,grand gaube,1,MU,1611965538,84,-20.0064,57.6608,25.56,0.89,25.56
69,69,800,atuona,5,PF,1611965546,76,-9.8000,-139.0333,25.96,8.77,25.96
144,144,800,morro agudo,0,BR,1611965562,61,-20.7314,-48.0578,26.00,1.49,26.00
192,192,800,cabo san lucas,1,MX,1611965570,57,22.8909,-109.9124,25.91,6.69,27.00
196,196,800,poum,4,NC,1611965573,70,-20.2333,164.0167,27.27,4.64,27.27
292,292,502,acoua,0,YT,1611965604,78,-12.7239,45.0582,27.00,2.06,27.00
316,316,500,cap malheureux,0,MU,1611965608,84,-19.9842,57.6142,25.56,0.89,25.56
472,472,800,sechura,1,PE,1611965632,63,-5.5569,-80.8222,25.76,7.79,25.76
540,540,800,acajutla,0,SV,1611965483,34,13.5928,-89.8275,28.00,2.57,28.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [60]:
hotel_df=ideal_temp.loc[:,["City", "Country", "Lat","Long"]]

In [61]:
# add hotel name as a column to clean df
hotel_df["Hotel Name"] = ""

In [62]:
hotel_df

,City,Country,Lat,Long,Hotel Name
11,carnarvon,AU,-24.8667,113.6333,
27,grand gaube,MU,-20.0064,57.6608,
69,atuona,PF,-9.8000,-139.0333,
144,morro agudo,BR,-20.7314,-48.0578,
192,cabo san lucas,MX,22.8909,-109.9124,
196,poum,NC,-20.2333,164.0167,
292,acoua,YT,-12.7239,45.0582,
316,cap malheureux,MU,-19.9842,57.6142,
472,sechura,PE,-5.5569,-80.8222,
540,acajutla,SV,13.5928,-89.8275,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
